# Computing Recency, Frequency, Monetary Value

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandasql import sqldf
from sklearn.preprocessing import scale
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram , linkage, cut_tree

In [2]:
pysqldf = lambda q: sqldf(q, globals())
%precision %.2f
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib widget

In [3]:
columns = ['customer_id', 'purchase_amount', 'date_of_purchase']
df = pd.read_csv('purchases.txt', header=None, sep='\t',
            names=columns)
df.sample(n=5, random_state=57)

,customer_id,purchase_amount,date_of_purchase
4510,8060,30.00,2014-12-24
17761,109180,50.00,2009-11-25
39110,9830,30.00,2007-06-12
37183,56400,60.00,2009-09-30
33705,41290,60.00,2007-08-21


In [4]:
# interpret the last column as datetime
df['date_of_purchase'] = pd.to_datetime(df['date_of_purchase'], 
                                        format='%Y-%m-%d')
# Extract year of purchase and save it as a column
df['year_of_purchase'] = df['date_of_purchase'].dt.year
df.head()

,customer_id,purchase_amount,date_of_purchase,year_of_purchase
0,760,25.00,2009-11-06,2009
1,860,50.00,2012-09-28,2012
2,1200,100.00,2005-10-25,2005
3,1420,50.00,2009-07-09,2009
4,1940,70.00,2013-01-25,2013


In [5]:
# Add a day_since column showing the difference between last purchase and a basedate
basedate = pd.Timestamp('2016-01-01')
df['days_since'] = (basedate - df['date_of_purchase']).dt.days

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51243 entries, 0 to 51242
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customer_id       51243 non-null  int64         
 1   purchase_amount   51243 non-null  float64       
 2   date_of_purchase  51243 non-null  datetime64[ns]
 3   year_of_purchase  51243 non-null  int64         
 4   days_since        51243 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 2.0 MB


In [7]:
df.head()

,customer_id,purchase_amount,date_of_purchase,year_of_purchase,days_since
0,760,25.00,2009-11-06,2009,2247
1,860,50.00,2012-09-28,2012,1190
2,1200,100.00,2005-10-25,2005,3720
3,1420,50.00,2009-07-09,2009,2367
4,1940,70.00,2013-01-25,2013,1071


Next we are going to compute the customers recency, frequency and average purchase amount. This part is a big tricky specially when it's done with pandas. 

The trick here is that the customer ID will only appear once for every customer. So even though we have 51,000 purchases we'll only have as many unique customer IDs as there are in the database.

Now, for each customer, we need to compute the minimum number of days between all of his or her purchases and january 1st, 2016. Of course, if we take the minimum number of days, then we are going to have the day of the last purchase, which is the very definition of **recency**.

Then for each customer we need to compute the frequency, which is basically how many purchases that customer has made.

 The asterisk here basically means anything in the data that is related to that customer we could have put any known variable here, it wouldn't change and then for the amount we going to compute the average of purchase amount for that specific customer ID and name that aggregate computation as amount. Now, the trick is that we'd like to make sure that each row only appears one for each customer. So we going to compute that from the data and group by one, meaning that everything here is going to be computed and grouped per customer ID

In [8]:
# Compute recency, frequency, and average purchase amount
q = """
        SELECT customer_id,
        MIN(days_since) AS 'recency',
        COUNT(*) AS 'frequency',
        AVG(purchase_amount) AS 'amount'
        FROM df GROUP BY 1"""
customers = sqldf(q)

In [9]:
customers

,customer_id,recency,frequency,amount
0,10,3829,1,30.00
1,80,343,7,71.43
2,90,758,10,115.80
3,120,1401,1,20.00
4,130,2970,2,50.00
...,...,...,...,...
18412,263820,1,1,10.00
18413,263870,135,1,50.00
18414,263880,34,1,20.00
18415,263890,5,1,54.00


In [10]:
customers.describe()

,customer_id,recency,frequency,amount
count,"18,417.00","18,417.00","18,417.00","18,417.00"
mean,"137,573.51","1,253.04",2.78,57.79
std,"69,504.60","1,081.44",2.94,154.36
min,10.00,1.00,1.00,5.00
25%,"81,990.00",244.00,1.00,21.67
50%,"136,430.00","1,070.00",2.00,30.00
75%,"195,100.00","2,130.00",3.00,50.00
max,"264,200.00","4,014.00",45.00,"4,500.00"


In [11]:
df.describe()

,customer_id,purchase_amount,year_of_purchase,days_since
count,"51,243.00","51,243.00","51,243.00","51,243.00"
mean,"108,934.55",62.34,"2,010.87","1,631.94"
std,"67,650.61",156.61,2.88,"1,061.08"
min,10.00,5.00,"2,005.00",1.00
25%,"57,720.00",25.00,"2,009.00",733.00
50%,"102,440.00",30.00,"2,011.00","1,500.00"
75%,"160,525.00",60.00,"2,013.00","2,540.00"
max,"264,200.00","4,500.00","2,015.00","4,016.00"


In [12]:
customers.hist(column='recency', bins=70);
plt.xlim(0,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.00, 4214.65)

In [13]:
customers.hist(column='frequency', bins=40);
plt.xlim(0,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.00, 30.00)

In [14]:
customers.hist(column='amount', bins=421);
plt.xlim(0,400)
plt.ylim(0,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.00, 5008.50)

# Preparing and Transforming Data


In [15]:
new_data = customers.copy()

In [16]:
new_data.head()

,customer_id,recency,frequency,amount
0,10,3829,1,30.00
1,80,343,7,71.43
2,90,758,10,115.80
3,120,1401,1,20.00
4,130,2970,2,50.00


In [17]:
new_data.set_index('customer_id', inplace=True)

In [18]:
new_data.head()

,recency,frequency,amount
customer_id,,,
10,3829,1,30.00
80,343,7,71.43
90,758,10,115.80
120,1401,1,20.00
130,2970,2,50.00


In [19]:
# As we have seen in the previous plots, the histogram of average purchase amounts
# is extremely skewed to the left. Which makes it unuseful for segmentation purpose
# So let's take the log of it
new_data['amount'] = np.log10(new_data['amount'])
new_data.hist(column='amount');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# Scaling using the sklearn scaling function
new_data = pd.DataFrame(scale(new_data), index=new_data.index, columns=new_data.columns)
new_data.head()

# Now the data is ready for being segmented and analyzed

,recency,frequency,amount
customer_id,,,
10,2.38,-0.61,-0.24
80,-0.84,1.44,0.89
90,-0.46,2.46,1.52
120,0.14,-0.61,-0.76
130,1.59,-0.27,0.43


# Running a Hierarchical Segmentation

In [21]:
# Compute distance metrics on standardized data
# This will likely generate an error on most machines
# R: d = dist(new_data) 
# since it generate a matrix with:
print(f'({new_data.shape[0]**2}, {new_data.shape[1]})')

(339185889, 3)


In [22]:
# so Let's take a 10% sample
sample = np.arange(0, 18417, 10)
sample[:10]

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [23]:
new_data_sample = new_data.iloc[sample]
new_data_sample.head()

,recency,frequency,amount
customer_id,,,
10,2.38,-0.61,-0.24
260,2.27,-0.61,-1.14
510,2.26,-0.61,-0.24
850,-1.14,2.46,-0.30
1040,-1.11,0.41,0.22


In [24]:
customers_sample = customers.iloc[sample].copy()
customers_sample.head()

,customer_id,recency,frequency,amount
0,10,3829,1,30.00
10,260,3710,1,15.00
20,510,3698,1,30.00
30,850,24,10,28.50
40,1040,56,4,42.50


In [25]:
# Compute the distance metrics on standardized data
d = pdist(new_data_sample)
d.shape

(1695561,)

In [26]:
# Perform hierarchical clustering on distance metrics
c = linkage(d, method='ward')

plt.figure(figsize=(10, 7))
plt.title("Customer Dendograms")

# Plot dendrogram
dend = dendrogram(c)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
c.shape

(1841, 4)

In [28]:
members = cut_tree(c, n_clusters=9)

In [29]:
members[:5]

array([[0],
       [1],
       [0],
       [2],
       [3]])

In [30]:
members.shape

(1842, 1)

In [37]:
groups, counts = np.unique(members, return_counts=True)
segments = dict(zip(groups, counts))

In [32]:
customers_sample['group'] = members.flatten()

In [33]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1842 entries, 0 to 18410
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  1842 non-null   int64  
 1   recency      1842 non-null   int64  
 2   frequency    1842 non-null   int64  
 3   amount       1842 non-null   float64
 4   group        1842 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 86.3 KB


In [35]:
clusters = customers_sample[['recency', 'frequency', 'amount', 'group']].groupby('group').mean()

In [36]:
clusters

,recency,frequency,amount
group,,,
0,"2,563.24",1.28,37.78
1,"2,684.29",1.31,16.26
2,193.65,10.62,42.02
3,162.10,2.40,41.11
4,"2,567.04",1.86,214.86
5,448.58,6.58,261.22
6,"1,143.69",4.47,29.85
7,922.83,1.28,22.43
8,799.34,1.40,87.43


Q1: What is the size of the largest segment?

Answer Q1:

In [44]:
max(segments.values())

317

Q2: What is the average purchase amount of the segment which contains, on average, the customers who have made their last purchase the most recently?

Answer Q2:

In [41]:
clusters.iloc[clusters['recency'].idxmin()]['amount']

41.10640522875818

In [48]:
customers_sample.set_index('customer_id', inplace=True)

KeyError: "None of ['customer_id'] are in the columns"

In [50]:
customers_sample.loc[260]['group'] == customers_sample.loc[5920]['group']

False

Looking at the average profile of segment 1, would you say that members of this segment are typically...(you'll have to choose the right proposition)

In [52]:
clusters

,recency,frequency,amount
group,,,
0,"2,563.24",1.28,37.78
1,"2,684.29",1.31,16.26
2,193.65,10.62,42.02
3,162.10,2.40,41.11
4,"2,567.04",1.86,214.86
5,448.58,6.58,261.22
6,"1,143.69",4.47,29.85
7,922.83,1.28,22.43
8,799.34,1.40,87.43
